In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# 分析対象データ（10尾の魚の体長）
np.random.seed(1)
fish = pd.Series(np.random.normal(loc=4, scale=0.8, size=10), name='length')
fish

0    5.299476
1    3.510595
2    3.577463
3    3.141625
4    4.692326
5    2.158769
6    5.395849
7    3.391034
8    4.255231
9    3.800504
Name: length, dtype: float64

----
## 点推定

- 標本平均を算出し、母平均の推定値とみなす
- 標本分散を算出し、母分散の推定値とみなす

In [3]:
mu = sp.mean(fish)
print('mean: {:.3f}'.format(mu))
sigma = sp.var(fish, ddof=1)
print('var:  {:.3f}'.format(sigma))

mean: 3.922
var:  1.009


----
## 区間推定

推定値に幅を持たせた推定方法のこと<br>
推定値の幅の計算には確率の考え方を用いる

区間推定に必要となる情報は以下の3つ

- 自由度（サンプルサイズ - 1）
- 標本平均
- 標本誤差

In [5]:
# 自由度
df = len(fish) - 1

# 標準誤差
se = sigma / sp.sqrt(len(fish))

In [10]:
# 信頼区間を算出
lower, upper = stats.t.interval(
    alpha=0.95,
    df=df,
    loc=mu,
    scale=sigma
)
print('Lo: {:.3f}'.format(lower))
print('Hi: {:.3f}'.format(upper))

Lo: 1.641
Hi: 6.204


In [11]:
# 標本標準偏差を 10 倍にしてみると、信頼区間の幅が広がる
se2 = (sigma*10) / sp.sqrt(len(fish))
stats.t.interval(alpha=0.95, df=df, loc=mu, scale=se2)

(-3.2922714426386253, 11.136846017348864)

In [13]:
# サンプルサイズを 10 倍にしてみると、信頼区間の幅は狭まる
df2 = len(fish) * 10 - 1
se3 = sigma / sp.sqrt(len(fish) * 10)
stats.t.interval(alpha=0.95, df=df2, loc=mu, scale=se3)

(3.7221738948570589, 4.1224006798531807)

#### 区間推定の結果の解釈

1. 真の母集団分布から標本を抽出する
1. 95% 信頼区間を計算する
1. 1と2の試行を繰り返す
1. すべての試行のうち、「真の母数」が信頼区間に含まれている割合が 95%

In [17]:
# 上記の内容をシミュレーションで確認する
# 信頼区間が母平均 4 を含んでいるか否かを求める
is_included = np.zeros(20000, dtype='bool')

np.random.seed(1)
norm_dist = stats.norm(loc=4, scale=0.8)

for i in range(20000):
    sample = norm_dist.rvs(size=10)
    df = len(sample) - 1
    mu = sp.mean(sample)
    sigma = sp.std(sample, ddof=1)
    se = sigma / sp.sqrt(len(sample))
    interval = stats.t.interval(alpha=0.95, df=df, loc=mu, scale=se)
    if (interval[0] <= 4 and interval[1] >= 4):
        is_included[i] = True

In [18]:
sp.sum(is_included) / len(is_included)

0.94799999999999995